In [ ]:
import xarray as xr
from climtas import nci
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pyart
import numpy as np
import datetime as dt
import pandas as pd


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [ ]:
#Note the run is from Feb 22 to March 7

In [ ]:
client = nci.GadiClient()
client

In [ ]:
#Load agcd
agcd = xr.open_dataset("/g/data/w40/asp561/agcd/agcd_v1_precip_total_r005_daily_2022.nc",chunks="auto")
agcd["time"] = (pd.to_datetime(agcd.time.values) + dt.timedelta(days=-1)).round("1D")

#Load agcd lsm
lsm = xr.open_dataset("/g/data/w40/asp561/agcd/mask_australia_0.05deg.nc").rename({"longitude":"lon","latitude":"lat"})
lsm["lon"] = lsm.lon.astype(np.float32)
lsm["lat"] = lsm.lat.astype(np.float32)

#mask agcd based on lsm
agcd = xr.where(lsm.landmask,agcd.precip,np.nan)

#mask aus2200 based on the agcd lsm
aus_prcp = xr.open_mfdataset("/g/data/hh5/tmp/WACI-Hackathon-2023/AUS2200/data/surf/10min/accum_ls_prcp_day_sum*",chunks="auto")
aus_prcp_lsm = lsm.interp({"lon":aus_prcp.longitude,"lat":aus_prcp.latitude},
                          method="nearest",method_non_numeric="pad").landmask.fillna(0)
aus_prcp = xr.where(aus_prcp_lsm,aus_prcp.accum_ls_prcp,np.nan)

In [ ]:
plt.figure(figsize=[14,6])

ax=plt.subplot(1,2,1,projection=ccrs.PlateCarree())
aus_prcp.sel({
    "time":slice("2022-03-01","2022-03-7"),
    "longitude":slice(147.9,154.6),
    "latitude":slice(-35.7,-24.9)}).sum("time").plot(
    ax=ax,cmap="pyart_Cat12",vmin=0,vmax=500)
ax.coastlines()
g=ax.gridlines(ls=":",draw_labels=True); g.top_labels=False; g.right_labels=False
plt.title("Aus2200")

ax=plt.subplot(1,2,2,projection=ccrs.PlateCarree())
agcd.sel({
    "time":slice("2022-03-01","2022-03-7"),
    "lon":slice(147.9,154.6),
    "lat":slice(-35.7,-24.9)}).sum("time").plot(
    ax=ax,cmap="pyart_Cat12",vmin=0,vmax=500)
ax.coastlines()

g=ax.gridlines(ls=":",draw_labels=True); g.top_labels=False; g.right_labels=False
plt.title("AGCD")

plt.suptitle("Week ending March 7",size=20)

In [ ]:
plt.figure(figsize=[14,6])

ax=plt.subplot(1,2,1,projection=ccrs.PlateCarree())
aus_prcp.sel({
    "time":slice("2022-02-23","2022-03-1"),
    "longitude":slice(147.9,154.6),
    "latitude":slice(-35.7,-24.9)}).sum("time").plot(
    ax=ax,cmap="pyart_Cat12",vmin=0,vmax=800)
ax.coastlines()
g=ax.gridlines(ls=":",draw_labels=True); g.top_labels=False; g.right_labels=False
plt.title("Aus2200")

ax=plt.subplot(1,2,2,projection=ccrs.PlateCarree())
agcd.sel({
    "time":slice("2022-02-23","2022-03-1"),
    "lon":slice(147.9,154.6),
    "lat":slice(-35.7,-24.9)}).sum("time").plot(
    ax=ax,cmap="pyart_Cat12",vmin=0,vmax=800)
ax.coastlines()

g=ax.gridlines(ls=":",draw_labels=True); g.top_labels=False; g.right_labels=False
plt.title("AGCD")

plt.suptitle("Week ending March 2",size=20)

In [ ]:
aus_prcp.sel({
    "time":slice("2022-02-23","2022-03-7"),
    "longitude":slice(147.9,154.6),
    "latitude":slice(-35.7,-24.9)}).mean(("latitude","longitude")).plot(marker="o",label="Aus2200")

agcd.sel({
    "time":slice("2022-02-23","2022-03-7"),
    "lon":slice(147.9,154.6),
    "lat":slice(-35.7,-24.9)}).mean(("lat","lon")).plot(marker="o",label="AGCD")

plt.legend()
plt.title("Mean rainfall over [-35.7,-24.9], [147.9,154.6]")

In [ ]:
aus_prcp.sel({
    "time":slice("2022-02-23","2022-03-7"),
    "longitude":slice(147.9,154.6),
    "latitude":slice(-35.7,-24.9)}).max(("latitude","longitude")).plot(marker="o",label="Aus2200")

agcd.sel({
    "time":slice("2022-02-23","2022-03-7"),
    "lon":slice(147.9,154.6),
    "lat":slice(-35.7,-24.9)}).max(("lat","lon")).plot(marker="o",label="AGCD")

plt.legend()
plt.title("Max rainfall over [-35.7,-24.9], [147.9,154.6]")

In [ ]:
#load radar data for brisbane

radar = xr.open_mfdataset([
    "/g/data/rq0/level_2/66/RAINRATE/66_20220223_rainrate.nc",
    "/g/data/rq0/level_2/66/RAINRATE/66_20220224_rainrate.nc",
    "/g/data/rq0/level_2/66/RAINRATE/66_20220225_rainrate.nc",
    "/g/data/rq0/level_2/66/RAINRATE/66_20220226_rainrate.nc",
    "/g/data/rq0/level_2/66/RAINRATE/66_20220227_rainrate.nc",
    "/g/data/rq0/level_2/66/RAINRATE/66_20220228_rainrate.nc",    
    "/g/data/rq0/level_2/66/RAINRATE/66_20220301_rainrate.nc",        
    "/g/data/rq0/level_2/66/RAINRATE/66_20220302_rainrate.nc"])

dt = (radar.time.values[-1] - radar.time.values[0])
total_hours = (((dt.astype('timedelta64[s]')))).tolist().days * 24 + \
                (((dt.astype('timedelta64[s]')))).tolist().seconds / 60 / 60


#TODO: Need to convert this to mm accumulated (is currently a sum of mm/hr at each time step)
radar_rain = radar.rainrate.sum("time")
plt.pcolormesh(radar.longitude.isel({"time":0}).values,radar.latitude.isel({"time":0}),radar_rain.values)
plt.colorbar()